# Import Libraries

In [1]:
from swarmintelligence import GeneticAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/GA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_gens': 100, 'mr': 0.2, 'cr': 0.6}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
ga_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                ga_evaluation_results['image_name'].append(image_name)
                ga_evaluation_results['thresholds'].append(threshold)
                ga_evaluation_results['fitness_function'].append(fitness_function)
                ga_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # GA
                    ga = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    # Evaluation Metrics
                    st = time.time()
                    populations, best_thresholds = ga.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(ga.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(ga.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                ga_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                ga_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                ga_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                ga_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                ga_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                ga_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                ga_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                ga_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                ga_evaluation_results['Regions'].append(mean_regions)
                ga_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                ga_evaluation_results['Fitness'].append(fitness_values)
                ga_evaluation_results['MSE'].append(list_mse)
                ga_evaluation_results['RMSE'].append(list_rmse)
                ga_evaluation_results['PSNR'].append(list_psnr)
                ga_evaluation_results['SSIM'].append(list_ssim)
                ga_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
12.139980128364584
12.201402853772239
12.169601845919091
12.155664760948264
12.163164679567387
12.130510960828019
11.93219788788164
12.201528333758798
12.200101622338444
12.184039841694116
12.16302374072171
12.185208660464074
12.147706648753697
12.201528333758798
12.14260554157437
12.179880115287382
12.070419190555437
12.196942506220442
12.198453778577912
12.174295244494859
11.941299360396407
12.201995832548945
12.189853715692365
12.141539725845666
12.114005036600627
12.152699266722655
12.197690205839601
12.144598984723835
11.93894665431327
12.187043986562122
Mean Fitness = 12.14493098149089
	 AirplaneF16.tiff | Threshold =  3
15.323017383755795
14.994544032250904
15.38598656835364
15.37615378412857
15.415206794960314
15.284125812276114
15.347608607743513
15.354382282847533
15.184479730726036
15.001954840101835
15.18575964021243
15.361497475463354
15.003647621469653
15.045872549509447
15.005836986623422
15.343342007534778
15.404164588025175
15.458592

20.945188843022464
21.098769554105303
20.325116619580818
20.294587813322984
Mean Fitness = 20.896416638816014
	 Mandrill.tiff | Threshold =  2
12.136539193664603
12.193919419592367
12.117639085891216
12.134753680967922
12.21378316452267
12.107301193303483
12.059991081325222
12.176106271895922
12.128232845873228
12.205997647551207
12.15436441796875
12.179518932007298
12.1011297762578
12.139724101663274
12.18071877362091
12.214370445183157
12.110546854850533
12.111148553778023
12.130319926930724
12.007198003433778
12.028119783606993
12.217644677056194
12.166808417560645
12.217096462806232
12.17359251526636
12.148518669781925
12.060007274122889
12.158723954773745
12.197845508665415
12.107426378592129
Mean Fitness = 12.142636233750487
	 Mandrill.tiff | Threshold =  3
15.049991509251926
14.838760528940782
14.999245489194236
15.258710662467841
15.037989806695851
15.127246536459985
14.85027705233183
14.773809018725292
15.093791600704742
15.173514787319306
15.121056537411626
15.184891357337982

19.873075223416308
20.408626068909715
20.04691149849415
20.853097296615434
20.309317556316643
20.17918216222694
20.45451324877749
19.759227919555904
20.483096252236102
20.734250266470646
19.724298850945914
20.270023371498674
Mean Fitness = 20.344920063632287


## Visualize Results using DataFrame

In [7]:
ga_evaluation_results_df =pd.DataFrame(ga_evaluation_results)
ga_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[80, 167]",0.990809,12.144931,2711.310299,52.007114,13.819740,0.759855,0.927243,"[[80, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0.6729016304016113, 0.6919074058532715, 0.688...","[12.139980128364584, 12.201402853772239, 12.16...","[3168.951934814453, 2740.1953201293945, 2597.6...","[56.29344486540554, 52.346874979595434, 50.967...","[13.121647088038833, 13.75298840590906, 13.985...","[0.7081502965667754, 0.7643454209458556, 0.773...","[0.9251647489067403, 0.9251232727516595, 0.931..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[77, 126, 177]",3.519595,15.251979,2166.228236,46.474885,14.799132,0.787851,0.944607,"[[77, 255, 255, 255, 255, 255, 255, 255, 255, ...","[1.671950340270996, 2.85688853263855, 4.090754...","[15.323017383755795, 14.994544032250904, 15.38...","[2165.229782104492, 2157.5986099243164, 2591.4...","[46.532029636633006, 46.44995812618475, 50.906...","[14.775763688138014, 14.791097072258161, 13.99...","[0.7633305224155906, 0.806830695495749, 0.7542...","[0.9440820157595198, 0.9470593297889138, 0.930..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[62, 101, 146, 184]",3.502496,17.884473,1839.392508,42.699724,15.565110,0.783118,0.954894,"[[101, 255, 255, 255, 184, 255, 255, 255, 255,...","[3.218076467514038, 3.071371078491211, 2.95876...","[17.281483135952264, 17.447893506193573, 17.62...","[2572.0237197875977, 863.7445449829102, 1642.4...","[50.71512318616211, 29.389531214071962, 40.526...","[14.02805391434791, 18.766950432831106, 15.975...","[0.7638464071122821, 0.7184309278392543, 0.774...","[0.9430740322727981, 0.9706746147722591, 0.956..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[55, 88, 121, 153, 188]",3.607843,20.295427,1690.051341,40.959332,15.917637,0.785613,0.959609,"[[88, 255, 255, 188, 188, 188, 255, 255, 255, ...","[2.7944774627685547, 2.534118890762329, 3.8291...","[20.54091156540638, 19.409580298681515, 20.590...","[1818.381275177002, 1992.102596282959, 1267.10...","[42.642482047566155, 44.63297655638664, 35.596...","[15.533954102853848, 15.137686593845693, 17.10...","[0.8139683671478122, 0.7777483116412981, 0.752...","[0.9591757600105502, 0.9467318175318519, 0.968..."
4,Lena.png,2,kapur_entropy,max,"[92, 161]",3.572263,12.219304,2683.334754,51.387834,13.978875,0.612296,0.878498,"[[255, 255, 255, 255, 255, 161, 255, 255, 255,...","[3.725905179977417, 3.88761568069458, 3.627422...","[12.320076749243725, 12.316259172716126, 12.10...","[2441.3724212646484, 3286.156711578369, 2239.3...","[49.41024611621206, 57.32500947735089, 47.3212...","[14.254463265702913, 12.963920904630555, 14.62...","[0.6131458826944337, 0.6114345361582778, 0.611...","[0.8932698662044296, 0.8768732714161078, 0.866..."
5,Lena.png,3,kapur_entropy,max,"[85, 134, 179]",3.373398,15.135601,1454.264846,37.775805,16.656868,0.683460,0.921100,"[[179, 179, 179, 179, 179, 179, 179, 179, 179,...","[3.9109251499176025, 3.6852474212646484, 3.226...","[14.905580448877796, 15.104599074214725, 14.83...","[1259.1655807495117, 1542.7925872802734, 1303....","[35.484723202379804, 39.27839848161166, 36.106...","[17.1299751711631, 16.247728173416576, 16.9791...","[0.6764527018766582, 0.6814020921712596, 0.660...","[0.9236015950355969, 0.9295590523253243, 0.903..."
6,Lena.png,4,kapur_entropy,max,"[73, 109, 146, 188]",3.806530,17.703903,1022.606697,31.803846,18.127728,0.714759,0.942503,"[[188, 188, 188, 188, 188, 188, 188, 188, 188,...","[3.809467315673828, 4.608579397201538, 3.96949...","[17.50976305193387, 17.74837911013934, 17.5364...","[1076.91752243042, 848.468189239502, 833.24946...","[32.8164215360301, 29.128477290093656, 28.8660...","[17.808979174816887, 18.844447964731746, 18.92...","[0.7173501457680104, 0.7161172218943627, 0.695...","[0.9505951432498992, 0.9488973661199056, 0.947..."
7,Lena.png,5,kapur_e

# Save results

In [8]:
pickle.dump(ga_evaluation_results_df, open('results/evaluation/GA/GA_kapur_df.pkl', 'wb'))
pickle.dump(ga_evaluation_results, open('results/evaluation/GA/GA_kapur_dict.pkl', 'wb'))